In [5]:
import pandas as pd
import zipfile
import numpy as np

# Libreria utilizada para descargar los datos.
import kaggle

### Descarga y extraccion de los datos.

Los datos fueron recolectados de https://www.gob.mx/sesnsp/acciones-y-programas/datos-abiertos-de-incidencia-delictiva, para su posterior descarga en Kaggle. 

In [6]:
!kaggle datasets download -d electricreel/mexicos-crime-2015-2023

with zipfile.ZipFile("mexicos-crime-2015-2023.zip", 'r') as file:
    file.extractall()

crimes = pd.read_csv("IDEFC_NM_dic23.csv", encoding = 'latin1')

crimes

mexicos-crime-2015-2023.zip: Skipping, found more recently modified local copy (use --force to force download)


,Año,Clave_Ent,Entidad,Bien jurídico afectado,Tipo de delito,Subtipo de delito,Modalidad,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
0,2015,1,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,3,0,2,1,1,1,2,1,2,2,2,1
1,2015,1,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma blanca,1,1,0,0,0,1,0,1,0,0,0,1
2,2015,1,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con otro elemento,0,0,2,2,3,2,0,1,2,0,0,0
3,2015,1,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,No especificado,2,0,0,1,0,0,0,0,0,0,0,0
4,2015,1,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio culposo,Con arma de fuego,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28219,2023,32,Zacatecas,Otros bienes jurídicos afectados (del fuero co...,Falsificación,Falsificación,Falsificación,15,7,11,10,6,9,9,2,14,16,8,2
28220,2023,32,Zacatecas,Otros bienes jurídicos afectados (del fuero co...,Contra el medio ambiente,Contra el medio ambiente,Contra el medio ambiente,0,1,0,0,1,2,0,0,1,0,1,0
28221,2023,32,Zacatecas,Otros bienes jurídicos afectados (del fuero co...,Delitos cometidos por servidores públicos,Delitos cometidos por servidores públicos,Delitos cometidos por servidores públicos,16,20,27,23,25,32,25,14,46,28,26,8
28222,2023,32,Zacatecas,Otros bienes jurídicos afectados (del fuero co...,Electorales,Electorales,Electorales,0,0,0,2,0,1,1,1,0,0,0,0


Podemos ver claramente como es que la tabla contiene demasiadas columnas, lo cual puede dificultar el analisis de datos con respecto el tiempo, ya que cada mes esta incluido en una columna. Sumado a eso, algunos de estos datos pueden ser considerados innecesarios para el analisis, por lo que deberan ser reemplazados/eliminados.

Corrigiendo los datos faltantes en la lista.

In [7]:
crimes.fillna(value = 0, inplace = True)
crimes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28224 entries, 0 to 28223
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Año                     28224 non-null  int64 
 1   Clave_Ent               28224 non-null  int64 
 2   Entidad                 28224 non-null  object
 3   Bien jurídico afectado  28224 non-null  object
 4   Tipo de delito          28224 non-null  object
 5   Subtipo de delito       28224 non-null  object
 6   Modalidad               28224 non-null  object
 7   Enero                   28224 non-null  int64 
 8   Febrero                 28224 non-null  int64 
 9   Marzo                   28224 non-null  int64 
 10  Abril                   28224 non-null  int64 
 11  Mayo                    28224 non-null  int64 
 12  Junio                   28224 non-null  int64 
 13  Julio                   28224 non-null  int64 
 14  Agosto                  28224 non-null  int64 
 15  Se

### Transformacion de los datos para una mejor clasificacion:

- Reemplazo de 'Violación simple' y 'Violación equiparada' con "Violacion" en Tipo de delito.
- "Feminicidio" ahora es catalogado como "Homicidio".
- 'Hostigamiento sexual', 'Acoso sexual' y 'Abuso sexual' son clasificados como 'Delito sexual'

In [8]:
crimes  = pd.read_csv('IDEFC_NM_dic23.csv', encoding = 'latin1')

crimes['Tipo de delito'] = crimes['Tipo de delito'].replace('Feminicidio', 'Homicidio')

delito_sexual = ['Hostigamiento sexual', 'Acoso sexual','Abuso sexual']

crimes['Tipo de delito'] = crimes['Tipo de delito'].replace(delito_sexual, 'Delito sexual')

violacion = ['Violación simple' , 'Violación equiparada']

crimes['Tipo de delito'] = crimes['Tipo de delito'].replace(violacion, 'Violación')

crimes.to_csv('IDEFC_NM_dic23.csv', encoding='latin1' , index = False)

Aqui se esta calculando el crimen total de cada mes para cada tipo de delito. Tambien se esta conviertiendo las columnas de los meses en filas, esto para facilitar en analisis del tiempo y la legibilidad.

In [9]:
meses = ['Enero', "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", 
         "Octubre", "Noviembre", "Diciembre"]

new_crimes = crimes.groupby(["Entidad", "Tipo de delito", "Año"])[meses].sum().reset_index()

melted = new_crimes.melt(id_vars = ["Entidad", "Tipo de delito", "Año"] , value_vars = meses, var_name = "Mes", 
                         value_name= "Total")

Para ampliar la vision de este analisis, se ha incluido los datos de numero de poblacion de cada estado (recolectados desde el sitio oficial de la INEGI) para poder calcular la tasa de criminalidad. Estos datos fueron incluidos en la misma tabla anterior:

In [10]:
poblacion = pd.read_excel('Poblacion_01.xlsx')

merged = pd.merge(melted, poblacion, on = ['Entidad'], how= "left")

merged["Poblacion"] = np.where(melted["Año"] < 2020, merged['2010'], merged['2020'])

merged = merged.drop(['2010', '2020'], axis = 1)
    
merged.to_csv("Crime_Tabla_Final.csv", encoding = 'latin1', index = False)


Aqui podemos ver el resultado final, una tabla mas legible y ordenada.

In [11]:
merged

,Entidad,Tipo de delito,Año,Mes,Total,Poblacion
0,Aguascalientes,Aborto,2015,Enero,0,1184996.0
1,Aguascalientes,Aborto,2016,Enero,1,1184996.0
2,Aguascalientes,Aborto,2017,Enero,0,1184996.0
3,Aguascalientes,Aborto,2018,Enero,2,1184996.0
4,Aguascalientes,Aborto,2019,Enero,0,1184996.0
...,...,...,...,...,...,...
124411,Zacatecas,Violencia familiar,2019,Diciembre,234,1490668.0
124412,Zacatecas,Violencia familiar,2020,Diciembre,244,1622138.0
124413,Zacatecas,Violencia familiar,2021,Diciembre,227,1622138.0
124414,Zacatecas,Violencia familiar,2022,Diciembre,236,1622138.0
